In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import cv2
import os
from sklearn.model_selection import train_test_split
import random

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
d="D:\Athena\images"
classes=["plastic", "non-plastic"]

data=[]
for c in classes:
    i=classes.index(c)
    path=os.path.join(d,c)
    for img in os.listdir(path):
        im=cv2.imread(os.path.join(path,img))
        im=cv2.resize(im,(224,224))
        data.append([im,i])
        
random.shuffle(data)

In [6]:
X=[]
Y=[]
for x,y in data:
    X.append(x)
    Y.append(y)

x=np.array(X)
y=np.array(Y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0, stratify=y)

X_train = x_train / 255.0
X_test = x_test / 255.0

In [7]:
m=tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=None,
    alpha=1.0,
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)

In [8]:
base_input=m.layers[0].input
base_output=m.layers[-2].output
final_output = layers.Dense(128)(base_output) 
final_output = layers.Activation('relu')(final_output) 
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7, activation='softmax')(final_output) 

In [9]:
new_model = tf.keras.Model(inputs = base_input, outputs = final_output)
new_model.compile(loss="sparse_categorical_crossentropy", optimizer = "SGD", metrics = ["accuracy"])
path_checkpoint = "D:\Athena"
directory_checkpoint = os.path.dirname(path_checkpoint)
callback = tf.keras.callbacks.ModelCheckpoint(filepath=path_checkpoint, save_weights_only=True, verbose=1)
history=new_model.fit(x_train, y_train, epochs = 100,validation_data=(x_test, y_test))
new_model.evaluate(x_test,y_test)
new_model.save("model.h5")

Epoch 1/100
16/16 [==============================] - 26s 1s/step - loss: 1.0252 - accuracy: 0.6376 - val_loss: 0.8697 - val_accuracy: 0.7480
Epoch 2/100
16/16 [==============================] - 22s 1s/step - loss: 0.3047 - accuracy: 0.8950 - val_loss: 1.0038 - val_accuracy: 0.4724
Epoch 3/100
16/16 [==============================] - 22s 1s/step - loss: 0.1551 - accuracy: 0.9683 - val_loss: 0.9727 - val_accuracy: 0.4724
Epoch 4/100
16/16 [==============================] - 22s 1s/step - loss: 0.0852 - accuracy: 0.9901 - val_loss: 0.9846 - val_accuracy: 0.4252
Epoch 5/100
16/16 [==============================] - 22s 1s/step - loss: 0.0538 - accuracy: 1.0000 - val_loss: 1.1738 - val_accuracy: 0.3465
Epoch 6/100
16/16 [==============================] - 22s 1s/step - loss: 0.0410 - accuracy: 1.0000 - val_loss: 1.3499 - val_accuracy: 0.3307
Epoch 7/100
16/16 [==============================] - 22s 1s/step - loss: 0.0240 - accuracy: 1.0000 - val_loss: 1.4459 - val_accuracy: 0.3307
Epoch 8/100
1

In [11]:
imtest=cv2.imread("D:/Athena/test/p1.jpg")
imtest=cv2.resize(imtest,(224,224))
imtest=np.array(imtest)
model = load_model("D:\Athena\model.h5")
a=model.predict(imtest)
print(a)
if(a==0):
    print("plastic")
elif(a==1):
    print("non-plastic")
else:
    print("error")

ValueError: in user code:

    File "c:\Users\rajit\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\rajit\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\rajit\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\rajit\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "c:\Users\rajit\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users